# MP2 - Deconvolution

----------------

# Contents:
- **2D Deconvolution using Richardson-Lucy (RL) Algorithm [45 points]**
    - Implement RL algorithm [30]
    - Apply and visualize (2D astro image) [5]
    - Find plausible PSF for a given noisy brain image, comment, and plot [10]
- **3D Convolution [35 points]**
    - Implement 3D convolution [30]
    - Compare implementation with scipy [5]
- **3D Deconvolution using RL Algorithm [20 points]**
    - Implement 3D RL algorithm [15]
    - Apply and visualize (3D brain volumes) [5]
----------------

# [Section] 2D Deconvolution using Richardson-Lucy (RL) Algorithm [45 points]

In [ ]:
!pip install scikit-image
!curl https://upload.wikimedia.org/wikipedia/commons/0/03/T1-weighted-MRI.png --output brain_MRI.png

## Put your ```convolution2d``` function from MP1 in the cell below


In [ ]:
def convolve2D(image, kernel, padding=0, strides=1):
    '''
    Returns the result Y = image * kernel, where * is the 2-D convolution operator.

        Parameters:
                image (ndarray): 2-D input image/signal
                kernel (ndarray): 2-D convolution kernel/signal

        Returns:
                Y (ndarray): 2-D convolution result similar to "same" mode in scipy/numpy.
    '''
    return

### [Exercise] Implement the Richardson-Lucy deconvolution algorithm [30]
You should use your `convolve2D` function in the implementation. 

**Hint:** Make sure to use 'same' padding so that the convolution result has the same shape after each iteration.

In [ ]:
from scipy.signal import convolve

def richardson_lucy(image, psf, num_iter=50, eps=1e-12):
    """
    Richardson-Lucy deconvolution.
    
    ----------
    input:
    image (Numpy.Array): Input degraded image (can be N dimensional).
    psf (ndarray): The point spread function.
    num_iter (int): Number of iterations. This parameter plays the role of regularisation. default to 50.
    eps (float): Optional. Value below which intermediate results become 0 to avoid division by small numbers.

    output: (Numpy.Array) The deconvolved image.
    """
    return

Now let's try your R-L algorithm. First, load an image in grayscale.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from scipy.signal import convolve2d as conv2

from skimage import color, data

rng = np.random.default_rng()

astro = color.rgb2gray(data.astronaut())

Now manually add convolutional optical noise to the image by convolving the image with a point spread function (psf). Then add poisson noise to it.


In [ ]:
psf = np.ones((5, 5)) / 25
astro = conv2(astro, psf, 'same')

# add Noise to Image
astro_noisy = astro.copy()
astro_noisy += (rng.poisson(lam=25, size=astro.shape) - 10) / 255.

### [Exercise] Now apply the Richardson Lucy deconvolution algorithm, and plot the original image, the image with noise, and the deconvolution result. [5]

In [ ]:
# Restore noisy image using Richardson-Lucy algorithm
deconvolved_RL = ...

# Plot 1) original image, 2) noisy image, 3) deconvolution result side-by-side

Now that we have seen the power of the Richardson Lucy algorithm with artificial/known noise, let's try it out on an image with real unknown noise.

In [ ]:
import PIL
brain = np.array(PIL.Image.open("brain_MRI.png"))

if len(brain.shape) > 2: # check if the picture has multiple color channels. If yes, convert to greyscale
    brain = color.rgb2gray(brain) 
    
brain = brain / 256.  # normalize the image pixel data to [0,1]
brain += 1E-12 # adding a small constant 1E-12 to avoid division-by-zero later down the line

### [Exercise] Since we do not know the real psf function in this case, we have to make a guess. Find the psf function which best models the real convolutional noise by trying different psf values in the cell below. [10]

In [ ]:
psf = ...
deconvolved_brain = ...

# visualize original and deconv output side-by-side 

**Q:** What did you observe when trying different point spread functions? Can you explain your observations?

**Your Answer:**

--------------------
# [Section] 3D Convolution [35 points]

Now let's apply the Richardson-Lucy algorithm in the 3-D space. 

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.signal import convolve as scipy_convolve

### [Exercise] Implement the 3D convolution [30]

First, implement the 3-D convolution function in the cell below. Your implementation must support arbitrary padding modes.

In [ ]:
def convolve3D(image, kernel, padding=0):
    '''
    convolve3D takes in the image array and the kernel array and convolves them
    with the padding scheme specified by parameter "padding"

    Input:
    image (Numpy.Array): 3-D image to be convolved
    kernel (Numpy.Array): 3-D kernel to be convolved
    padding (int): padding mode. default is 0

    Output:
    result (Numpy.Array): result of the convolution between image and kernel
    '''

    return

### [Exercise] Test your 3-D convolution function in the cell below. [5]

In [ ]:
A = np.random.randint(0, 100, size=(30, 10, 18))
B = np.random.randint(0, 100, size=(5, 5, 3))

print("Correct (Scipy) Result:")
scipy_result = scipy_convolve(...)
print(np_result)

print()

print("Your Answer:")
your_result = ....
print(your_result)

print("\nYour result is {}".format("correct!" if np.array_equal(
    scipy_result, your_result) else "incorrect."))

You are encouraged to try different kernel sizes and padding modes to ensure your implementation is robust and general.

--------------

# [Section] 3D Deconvolution using RL Algorithm [20 points]

### [Exercise] Now with the 3D convolution function, implement the 3D Richardson-Lucy deconvolution algorithm. If your 3D convolution function does not work, use a library implementation instead. [15]

In [ ]:
def richardson_lucy_3d(image, psf, padding, num_iter=30, eps=1e-12):
    """Richardson-Lucy deconvolution.
    Parameters
    ----------
    image : ndarray
      Input degraded image (can be N dimensional).
    psf : ndarray
      The point spread function.
    eps: float, optional
      Value below which intermediate results become 0 to avoid division
      by small numbers.
    padding: int
      Value depends on size of psf used. Use the padding size in the "same" padding mode.
    num_iter : int, optional
      Number of iterations. This parameter plays the role of
      regularisation.
    Returns
    -------
    im_deconv : ndarray
      The deconvolved image.
    """
    return im_deconv

In [ ]:
import nibabel as nib
from skimage import color

!wget https://github.com/vb100/Visualize-3D-MRI-Scans-Brain-case/raw/master/data/images/BRATS_001.nii.gz

In [ ]:
image_path = "BRATS_001.nii.gz"
image_obj = nib.load(image_path)
image_data = image_obj.get_fdata()
type(image_data)
print(image_data.shape)
image_data_by_channel = np.array([image_data[:, :, :, i] for i in range(4)])
print(image_data_by_channel.shape)

The amount of computation in 3D deconvolution / convolution is exponentially higher than in the 2D space. To save some time, let's shrink the image by downsampling it to a factor of 0.7 in the cell below. You may choose different factors to test your implementation faster.

In [ ]:
from scipy.ndimage import zoom

image_data_by_channel = np.array(
    [zoom(channel, (0.7, 0.7, 0.7)) for channel in image_data_by_channel])

In [ ]:
rng = np.random.default_rng()

psf = np.ones((5, 5, 5)) / 125

convolved_by_channel = [scipy_convolve(
    channel_slice, psf) for channel_slice in image_data_by_channel]

noisy_by_channel = convolved_by_channel.copy()

noisy_by_channel = [channel_slice + (rng.poisson(lam=125, size=channel_slice.shape) - 10 / 255)
                    for channel_slice in noisy_by_channel]

### [EXERCISE] Finally, let's apply your 3-D Richardson-Lucy deconvolution implementation to the noisy volumetric brain image.[5]

**(Warning - num_iter=1 in the cell below may take 3-6mins to run!)**

In [ ]:
num_iter = ...

deconvolved_result = ...

In [ ]:
def display_deconvolution_comparison(layer_idx, channel_slice_idx):
    '''
    Display the original, noisy, and deconvolved image side-by-side given a particular 2D slice of the 4D brain image dataset.
    '''
    return

layer_idx = ...
channel_slice_idx = ...
display_deconvolution_comparison(layer_idx, channel_slice_idx)

-----------